In [2]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

from EDA_script_original2 import *


In [9]:
X_train, X_val, y_train, y_val = train_test_split(df_train, df_y, test_size=0.3, random_state=42)

# Define a range of values for the hyperparameters to search through
param_grid = {
    'num_leaves': [10,30,40],
    'learning_rate': [0.04],
    'n_estimators': [500],
    'max_depth': [7],
    'lambda_l1': [0.04],
    'lambda_l2': [0.01],
    'min_data_in_leaf': [10],
    'min_gain_to_split': [0.02],
    'feature_fraction': [0.1],
    'bagging_fraction': [0.1]
}

In [11]:
# Initialize an empty dictionary to store the best hyperparameters and their values
best_hyperparameters = {}
best_values = {}
 
# Initialize the LightGBM classifier
lgb_classifier = lgb.LGBMClassifier(objective='multiclass', num_class=5, boosting_type='gbdt', metric = 'multi_logloss', 
                                    early_stopping_rounds=10, verbosity = 2)
 

# Initialize GridSearchCV for hyperparameters
grid_search = GridSearchCV(estimator=lgb_classifier, param_grid=param_grid,
                           scoring='neg_log_loss', cv=5)
 

# Fit the model to the training data to search for the best hyperparameters
grid_search.fit(X_train, np.ravel(y_train))
 

# Get the best hyperparameters and their values
best_params = grid_search.best_params_
best_hyperparameters = list(best_params.keys())
best_values = list(best_params.values())

[LightGBM] [Warning] feature_fraction is set=0.1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_gain_to_split is set=0.02, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.02
[LightGBM] [Warning] lambda_l1 is set=0.04, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.04
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] lambda_l2 is set=0.01, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] feature_fraction is set=0.1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1
[LightGBM] [Warning] min_data_in_leaf is s

ValueError: 
All the 15 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\sklearn.py", line 967, in fit
    super().fit(X, _y, sample_weight=sample_weight, init_score=init_score, eval_set=valid_sets,
  File "c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\sklearn.py", line 748, in fit
    self._Booster = train(
  File "c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\en

In [ ]:
best_model = lgb.LGBMClassifier(**best_params)
best_model.fit(X_train, y_train)

NameError: name 'best_params' is not defined

In [ ]:
y_pred = best_model.predict(df_test)
 
# Calculate accuracy
accuracy = log_loss(df_test, y_pred)
lgb.plot_metric(best_model)
print('Best hyperparameters:', best_hyperparameters)
print('Best values:', best_values)
print(f'Accuracy with best hyperparameters: {accuracy:.4f}')